In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pandas.api.types import is_numeric_dtype
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, f1_score, roc_curve
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("hospital_readmission_decision_tree_dataset.csv")

In [4]:
df.head()

,Age,Blood_Pressure,BMI,Diabetes,Days_Admitted,Previous_Visits,Chronic_Disease,Readmitted
0,69,188.0,NaN,No,5,2,Yes,No
1,32,100.0,NaN,Yes,5,2,Yes,No
2,89,102.0,23.0,Yes,20,0,No,No
3,78,186.0,23.4,Yes,15,2,No,No
4,38,198.0,29.0,No,18,1,No,No
